In [30]:
import requests
from bs4 import BeautifulSoup
import time
import numpy as np
import pandas as pd
import random
from itertools import cycle
from requests.exceptions import ProxyError
from lxml.html import fromstring
import ast
with open('../.env', 'r') as f:
    key = f.read()

In [3]:
#proxies
proxie_file = 'proxies.txt'
try:
    with open(proxie_file) as f:
        proxies = f.readlines()
        proxy_pool = cycle(proxies)
except:
    pass

In [160]:
    def get_proxies():
        proxie_file = 'proxies.txt'
        try:
            with open(proxie_file) as f:
                proxies = f.readlines()
            return set(proxies)
        except:
            pass

In [152]:
    def get_proxies2():
        url = 'https://free-proxy-list.net/'
        headers = {}
        agent = ua_random()
        headers['user-agent'] = agent
        response = requests.get(url, headers = headers)
        parser = fromstring(response.text)
        proxies = set()
        for i in parser.xpath('//tbody/tr')[:10]:
            if i.xpath('.//td[7][contains(text(),"yes")]'):
                #Grabbing IP and corresponding PORT
                proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
                proxies.add(proxy)
        return proxies

In [5]:
   #agents
def ua_random():
        random_ua = ''
        ua_file = 'agents.txt'
        try:
            with open(ua_file) as f:
                lines = f.readlines()
            if len(lines) > 0:
                random_ua = random.choice(lines)
                random_ua = random_ua.split('\n')
        except Exception as ex:
            print('Exception in ua_random')
            print(str(ex))
        finally:
            return random_ua[0] 

In [6]:
def quotes_parser_refs(content):
    soup1 = BeautifulSoup(content)
    
    print(soup1.prettify())

In [158]:
def scrape_url(url, headers):
    proxies = get_proxies()
    proxy_pool = cycle(proxies)
    flag = False
    print(headers)
    while flag == False:
        proxy = next(proxy_pool)
        #print(proxy)
        try:
            r = requests.get(url, headers = headers, proxies={"http": proxy})
            r.raise_for_status()
            #print('hola2:', BeautifulSoup(r.content).prettify())
            flag = True
            return BeautifulSoup(r.content)
            #quotes_parser_refs(r)
        except requests.exceptions.HTTPError as e_http:
            pass
            #result = "Error http:" + str(e_http)
        #except ProxyError:
            #print('Proxy Error')
            #pass
        except requests.exceptions.RequestException as e:
            pass
            #result = "Error Desconocido: " + str(e)
    if flag == False:
        # none of the proxies worked
        raise ProxyError
            

In [144]:
def parser_page(content):
    anuncios = content.find_all('div', 'information')
    pisos = []
    for piso in anuncios:
        items = {}
        #print(piso.find('div', 'price').contents[0].strip().split(' ')[0])
        items['location1'] = piso.find('a', 'anuncioLink').string.replace('Piso en ', '')
        items['location2'] = str(piso.find('div', 'location').string).strip()
        items['price'] = piso.find('div', 'price').contents[0].strip().split(' ')[0]
        items['description'] = str(piso.find('div', 'description').string).strip()
        chars = piso.find('div', 'characteristics').find_all('div', 'item')
        #print(chars)
        for char in chars:
            #print('CONTENTS:', char.contents[0].strip())
            #print('€/m²' in char.contents[0].strip())
            #print('ICOOOONOOOOOO:',char.find('span', 'icoBed') != None)

            if '€/m²' in char.contents[0].strip():
                items['price_m2'] = char.contents[0].strip().split(' ')[0].strip()

            elif 'm²' in char.contents[0]:
                items['size'] = char.contents[0].strip().split(' ')[0]

            elif char.find('span', 'icoBed') != None:
                items['rooms'] = int(char.contents[0].strip().split(' ')[0])
                #print(items)

            elif char.find('span', 'icoBath') != None:
                items['bathrooms'] = int(char.contents[0].strip().split(' ')[0])

            elif 'planta' in char.contents[0]:
                items['floor'] = char.contents[0].strip().split(' ')[0]

            elif 'Bajo' in char.contents[0]:
                items['floor'] = 0
        
        pisos.append(items)
    return pisos

In [122]:
URL_general = 'https://www.pisos.com/venta/piso-madrid/%s/'
range_to_scrape = range(1, 3)
headers = {'Accept-Encoding': 'gzip, deflate, br',
          'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7',
          'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
          'Cache-Control': 'max-age=0',
          'Upgrade-Insecure-Requests': '1',
          'Refereer': 'https://www.pisos.com/venta/piso-madrid/',
          'Host': 'www.pisos.com'}

def kickstart(URL_general, range_to_scrape, headers):
    datos_pisos = []
    for i in range_to_scrape:
        agent = ua_random()
        headers['user-agent'] = agent
        content = scrape_url(URL_general % i, headers)
        datos_pisos += parser_page(content)
        time.sleep(random.randint(2, 5))
    return datos_pisos

In [123]:
resultado1_2 = kickstart(URL_general, range_to_scrape, headers)

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'UCWEB/2.0 (MIDP-2.0; U; Adr 5.1.1; en-US; LS-5010) U2/1.0.0 UCBrowser/10.7.6.805 U2/1.0.0 Mobile'}
85.196.147.146:48140
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; Touch; rv:11.0) like Gecko'}
85.196.147.146:4814

In [125]:
len(resultado1_2)

60

In [161]:
resultado3_150 = kickstart(URL_general, range(3, 150), headers)

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 9_3_2 like Mac OS X) AppleWebKit/601.1 (KHTML, like Gecko) CriOS/51.0.2704.104

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.11; rv:48.0) Gecko/20100101 Firefox/48.0'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
{'Accept-Encoding': 'gzip, d

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Linux; Android 4.4.2; ZTE V831W Build/KOT49H) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.81 Mobile Safari/537.36'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Ch

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.130 Safari/537.36'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727)'}
{'Accept-Encoding': 'gzip, 

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 9_3_5 like Mac OS X) AppleWebKit/601.1.46 (KHTML, like Gecko) Mobile/13G36'}
{'Accept-Encoding': 'gzip, defl

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:46.0) Gecko/20100101 Firefox/46.0'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.1; Trident/4.0; GTB7.5; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1650.57 Safari/537.36'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36'}
{'

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (compatible; phpservermon/3.1.0; +http://www.phpservermonitor.org)'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 6.1; rv:48.0) Gecko/20100101 Firefox/48.0'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_5_8) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.90 Safari/537.1'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:47.0) Gecko/20100101 Firefox/47.0'}
{'Accept-Encoding': 'gz

In [9]:
'''
with requests.session() as r:
    page = None

    for _ in range(len(d)):
        d.rotate(-1)
        try:
            page = r.get(line.rstrip(), proxies=d[0])
        except ProxyError:
            # ignore proxy exception, move to next proxy
            pass
        else:
            # success, break loop
            break

    if page is None:
        # none of the proxies worked
        raise ProxyError
'''

'\nwith requests.session() as r:\n    page = None\n\n    for _ in range(len(d)):\n        d.rotate(-1)\n        try:\n            page = r.get(line.rstrip(), proxies=d[0])\n        except ProxyError:\n            # ignore proxy exception, move to next proxy\n            pass\n        else:\n            # success, break loop\n            break\n\n    if page is None:\n        # none of the proxies worked\n        raise ProxyError\n'

In [129]:
len(resultado3_150)

1172

In [167]:
total = resultado3_150 + resultado150_349 + resultado1_2 + resultado350_520
with open('house_data2.txt', 'w') as f:
    for item in total:
        f.write("%s\n" % item)

In [163]:
resultado150_349 = kickstart(URL_general, range(150, 350), headers)

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_5_8) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.90 Safari/537.1'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/53

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/601.7.7 (KHTML, like Gecko) Version/9.1.2 Safari/601.7.7'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Sogou web spider/4.0(+http://www.sogou.com/docs/help/webmasters.htm#07)'}
{'Accept-Encoding': 'gzip

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'UCWEB/2.0 (MIDP-2.0; U; Adr 4.0.4; en-US; GT-S7562) U2/1.0.0 UCBrowser/10.6.5.726 U2/1.0.0 Mobile'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
{

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Linux; Android 4.4.2; HTC Desire 526GPLUS dual sim Build/KOT49H) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.81 Mobile Safari/537.36'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'SAMSUNG-GT-S5263/S5263DDKA5 SHP/VPP/R5 Dolfin/2.0 NexPlaye

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'CITRIXRECEIVER'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 6.1; rv:48.0) Gecko/20100101 Firefox/48.0'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/x

...

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Linux; Android 5.1.1; LS-5015 Build/LMY47V) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.83 Mobile Safari/537.36'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:48.0) Gecko/20100101 Firefox/48.0'}
{'Ac

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.1; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0)'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, l

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Linux; Android 5.1.1; SM-T377R4 Build/LMY47X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.98 Safari/537.36'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1490

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.63 Safari/537.36'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:42.0) Gecko/20100101 Firefox/42.0'}
{'Accept-Encoding': 'gzip, defla

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (iPad; CPU OS 9_3_5 like Mac OS X) AppleWebKit/601.1.46 (KHTML, like Gecko) Version/9.0 Mobile/13G36 Safari/601.1'}
{'Accept-Encoding':

In [132]:
len(resultado150_349 )

4

In [141]:
def kickstart_test(URL_general, headers):
    agent = ua_random()
    headers['user-agent'] = agent
    content = scrape_url(URL_general, headers)
    return content

In [142]:
test_349 = kickstart_test('https://www.pisos.com/venta/piso-madrid/349/', headers)

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:48.0) Gecko/20100101 Firefox/48.0'}


In [143]:
print(test_349.prettify())

<!DOCTYPE html>
<html lang="es">
 <head>
  <meta charset="utf-8"/>
  <title>
   Pisos y apartamentos en Madrid - pisos.com
  </title>
  <link href="/faviconnew.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="/Images/assets/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="/manifest.json" rel="manifest"/>
  <meta content="#ffffff" name="theme-color"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="907BFBB940FF15950271AB090B57DCCB" name="msvalidate.01"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <link href="https://www.pisos.com/ca/venta/piso-madrid/349/" hreflang="ca" rel="alternate"/>
  <link href="https://www.pisos.com/en/venta/piso-madrid/349/" hreflang="en" rel="alternate"/>
  <meta content="noindex, follow" name="robots"/>
  <meta content="15.670 pisos y apartamentos en venta en Madrid. Oferta de profesionales y particulares desde 499000 euros en Madrid. ¡Un piso increíble está espe

[{'location1': 'Mejorada del Campo',
  'location2': 'Mejorada del Campo',
  'price': '104.400',
  'description': 'Vivienda de 3 dormitorios y 1 baños, cerca del centro de Mejorada del Campo. Vivienda situada en la segun...',
  'size': '71',
  'rooms': 3,
  'bathrooms': 1,
  'price_m2': '1.470'},
 {'location1': 'calle Legionario Poeta Queixa',
  'location2': 'San Isidro (Distrito Carabanchel. Madrid Capital)',
  'price': '137.000',
  'description': 'Estupenda vivienda en la zona más tranquila del barrio de San Isidro, muy próxima al parque del mismo nombre y c...',
  'size': '57',
  'rooms': 2,
  'bathrooms': 1,
  'floor': '3ª',
  'price_m2': '2.403'},
 {'location1': 'calle Sierra del Valle',
  'location2': 'Portazgo (Distrito Puente de Vallecas. Madrid Capital)',
  'price': '95.000',
  'description': 'Altura piso 1º, piso superficie total 62 m², superficie útil 57 m², hab. individuales: 1, habitaciones dobles: 1...',
  'size': '62',
  'rooms': 2,
  'bathrooms': 1,
  'floor': '1ª',
  'p

In [166]:
len(resultado350_520)

4110

In [153]:
get_proxies2()

set()

In [165]:
resultado350_520 = kickstart(URL_general, range(350, 521), headers)

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 5.1; rv:48.0) Gecko/20100101 Firefox/48.0'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Linux; Android 4.3; SHV-E250S Build/JSS15J) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.81 Mobile Safari/537.36'}
{'Accept-En

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 9_3_2 like Mac OS X) AppleWebKit/601.1.46 (KHTML, like Gecko) Mobile/13F69'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 5.1; rv:49.0) Gecko/20100101 Firefox/49.0'}
{'Accept-Encoding': 'gzip, deflate, br

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Linux; Android 6.0.1; SM-G935V Build/MMB29M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.98 Mobile Safari/537.36'}
{'Accept-En

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.87 Safari/537.36'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (compatible; phpservermon/3.1.0; +http://www.phpservermonitor.org)'}
{'Accept-Encoding': 'gzip

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.10240'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'facebookexternalhit/1.1 (+http://www.facebook.com/externalhit_uatext.php)'}
{'Accept-E

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) coc_coc_browser/50.2.173 Chrome/44.2.2403.173 Safari/537.36'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chro

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Mobile; Windows Phone 8.1; Android 4.0; ARM; Trident/7.0; Touch; rv:11.0; IEMobile/11.0; NOKI

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows Phone 10.0; Android 6.0.1; Microsoft; Lumia 950 XL) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Mobile Safari/537.36 Edge/14.14393'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'SAMSUNG-GT-B7300/1.0 (compatible; MSIE 6.0; Windo

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG-SM-G920A Build/MMB29K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.98 Mobile Safari/537.36'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Ge

{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (compatible; Baiduspider/2.0; +http://www.baidu.com/search/spider.html)'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-US,en;q=0.9,es-ES;q=0.8,es;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1', 'Refereer': 'https://www.pisos.com/venta/piso-madrid/', 'Host': 'www.pisos.com', 'user-agent': 'Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)'}
{'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language'

In [170]:
data = pd.DataFrame(total)

In [171]:
data.head()

,bathrooms,description,floor,location1,location2,price,price_m2,rooms,size
0,1.0,Coslada junto al recinto ferial. Vivienda de 7...,4ª,Recinto Ferial,Casco Antiguo (Coslada),226.000,2.897,2.0,78
1,1.0,"Bajos de 2 dormitorios 50m2, totalmente refor...",0,"calle Isletas, nº 1",Zona Estación (Coslada),98.500,1.790,2.0,55
2,1.0,Características del edificio:\n\n- Construido ...,NaN,"calle Mazo, nº 6",Pueblo Nuevo (Distrito Ciudad Lineal. Madrid C...,129.000,1.842,3.0,70
3,1.0,REF: 12130 Vivienda exterior localizada en una...,NaN,Vallecas,Palomeras Bajas (Distrito Puente de Vallecas. ...,220.000,3.098,3.0,71
4,1.0,"Excelente piso reformado en venta, de dos dorm...",0,calle Gomez Acebo,San Andrés (Distrito Villaverde. Madrid Capital),148.000,2.144,2.0,69


In [173]:
data.shape

(12510, 9)

In [181]:
#geocode_result = gmaps.geocode('1600 Amphitheatre Parkway, Mountain View, CA')

geocode_result = gmaps.geocode('Recinto Ferial, Casco Antiguo (Coslada)')

Timeout: 

In [177]:
data['location'] = data['location1'] + ', ' + data['location2'] 

In [179]:
data.head(3)

,bathrooms,description,floor,location1,location2,price,price_m2,rooms,size,location
0,1.0,Coslada junto al recinto ferial. Vivienda de 7...,4ª,Recinto Ferial,Casco Antiguo (Coslada),226.000,2.897,2.0,78,"Recinto Ferial, Casco Antiguo (Coslada)"
1,1.0,"Bajos de 2 dormitorios 50m2, totalmente refor...",0,"calle Isletas, nº 1",Zona Estación (Coslada),98.500,1.790,2.0,55,"calle Isletas, nº 1, Zona Estación (Coslada)"
2,1.0,Características del edificio:\n\n- Construido ...,NaN,"calle Mazo, nº 6",Pueblo Nuevo (Distrito Ciudad Lineal. Madrid C...,129.000,1.842,3.0,70,"calle Mazo, nº 6, Pueblo Nuevo (Distrito Ciuda..."


In [27]:
def get_position(x):
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address='
    try:
        r = requests.get(url + '+'.join(x.split(' ')) + '&key=' + key)
        datos = r.json()
        location = datos['results'][0]['geometry']['location']
        #time.sleep(2)
        return location
    except:
        return None
    
#r =https://maps.googleapis.com/maps/api/geocode/json?address=1600+Amphitheatre+Parkway,+Mountain+View,+CA&key=

In [ ]:
#data['position'] = data['location'].apply(lambda x: get_position(x))

In [15]:
with open('house_data2.txt', 'r') as f:
    data_list = ast.literal_eval(f.read())

SyntaxError: invalid syntax (<unknown>, line 2)

In [9]:
data_re = pd.DataFrame(data_list)
data_re.head()

,0
0,"{'location1': 'Recinto Ferial', 'location2': '..."
1,"{'location1': 'calle Isletas, nº 1', 'location..."
2,"{'location1': 'calle Mazo, nº 6', 'location2':..."
3,"{'location1': 'Vallecas', 'location2': 'Palome..."
4,"{'location1': 'calle Gomez Acebo', 'location2'..."


In [ ]:
with open(filepath) as fp:  
   line = fp.readline()
   cnt = 1
   while line:
       print("Line {}: {}".format(cnt, line.strip()))
       line = fp.readline()
       cnt += 1

In [17]:
with open('house_data2.txt') as f:
    data_list = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
data_list = [x.strip() for x in data_list] 

In [20]:
data_list[0:5]

[{'location1': 'Recinto Ferial',
  'location2': 'Casco Antiguo (Coslada)',
  'price': '226.000',
  'description': 'Coslada junto al recinto ferial. Vivienda de 78 m²., consta de dos dormitorios con a/e. (dormitorio principal co...',
  'size': '78',
  'rooms': 2,
  'bathrooms': 1,
  'floor': '4ª',
  'price_m2': '2.897'},
 {'location1': 'calle Isletas, nº 1',
  'location2': 'Zona Estación (Coslada)',
  'price': '98.500',
  'description': 'Bajos de 2 dormitorios 50m2,  totalmente reformados, amueblados con un diseño muy moderno a un precio muy accesi...',
  'size': '55',
  'rooms': 2,
  'bathrooms': 1,
  'floor': 0,
  'price_m2': '1.790'},
 {'location1': 'calle Mazo, nº 6',
  'location2': 'Pueblo Nuevo (Distrito Ciudad Lineal. Madrid Capital)',
  'price': '129.000',
  'description': 'Características del edificio:\n\n- Construido en 1965, fachada enfoscada, edificio rehabilitado con la inspección ...',
  'size': '70',
  'rooms': 3,
  'bathrooms': 1,
  'price_m2': '1.842'},
 {'location1': '

In [19]:
data_list = [ast.literal_eval(x) for x in data_list] 

In [21]:
data_re = pd.DataFrame(data_list)
data_re.head()

,bathrooms,description,floor,location1,location2,price,price_m2,rooms,size
0,1.0,Coslada junto al recinto ferial. Vivienda de 7...,4ª,Recinto Ferial,Casco Antiguo (Coslada),226.000,2.897,2.0,78
1,1.0,"Bajos de 2 dormitorios 50m2, totalmente refor...",0,"calle Isletas, nº 1",Zona Estación (Coslada),98.500,1.790,2.0,55
2,1.0,Características del edificio:\n\n- Construido ...,NaN,"calle Mazo, nº 6",Pueblo Nuevo (Distrito Ciudad Lineal. Madrid C...,129.000,1.842,3.0,70
3,1.0,REF: 12130 Vivienda exterior localizada en una...,NaN,Vallecas,Palomeras Bajas (Distrito Puente de Vallecas. ...,220.000,3.098,3.0,71
4,1.0,"Excelente piso reformado en venta, de dos dorm...",0,calle Gomez Acebo,San Andrés (Distrito Villaverde. Madrid Capital),148.000,2.144,2.0,69


In [22]:
data_re['location'] = data_re['location1'] + ', ' + data_re['location2'] 

In [23]:
data_re.head()

,bathrooms,description,floor,location1,location2,price,price_m2,rooms,size,location
0,1.0,Coslada junto al recinto ferial. Vivienda de 7...,4ª,Recinto Ferial,Casco Antiguo (Coslada),226.000,2.897,2.0,78,"Recinto Ferial, Casco Antiguo (Coslada)"
1,1.0,"Bajos de 2 dormitorios 50m2, totalmente refor...",0,"calle Isletas, nº 1",Zona Estación (Coslada),98.500,1.790,2.0,55,"calle Isletas, nº 1, Zona Estación (Coslada)"
2,1.0,Características del edificio:\n\n- Construido ...,NaN,"calle Mazo, nº 6",Pueblo Nuevo (Distrito Ciudad Lineal. Madrid C...,129.000,1.842,3.0,70,"calle Mazo, nº 6, Pueblo Nuevo (Distrito Ciuda..."
3,1.0,REF: 12130 Vivienda exterior localizada en una...,NaN,Vallecas,Palomeras Bajas (Distrito Puente de Vallecas. ...,220.000,3.098,3.0,71,"Vallecas, Palomeras Bajas (Distrito Puente de ..."
4,1.0,"Excelente piso reformado en venta, de dos dorm...",0,calle Gomez Acebo,San Andrés (Distrito Villaverde. Madrid Capital),148.000,2.144,2.0,69,"calle Gomez Acebo, San Andrés (Distrito Villav..."


In [55]:
data_re.to_csv('house_data_total.csv',  index = None, header=True)

In [56]:
data_re.to_csv('house_data_total_backup.csv', index = False)

In [ ]:
dict_pos = {}
for location in data_re['location'][:100]:
    dict_pos[location] = get_position(location)

In [43]:
dict_pos2 = {}
for location in data_re['location'][100:5000]:
    dict_pos2[location] = get_position(location)

In [39]:
def get_value_from_dict(dict, x):
    try:
        return dict[x]
    except:
        return None

data_re['position'] = data_re.loc[:100, 'location'].apply(lambda x: get_value_from_dict(dict_pos, x))

In [38]:
dict_pos

{'Recinto Ferial, Casco Antiguo (Coslada)': {'lat': 40.43040939999999,
  'lng': -3.5578887},
 'calle Isletas, nº 1, Zona Estación (Coslada)': {'lat': 40.4260459,
  'lng': -3.5651646},
 'calle Mazo, nº 6, Pueblo Nuevo (Distrito Ciudad Lineal. Madrid Capital)': {'lat': 40.43073589999999,
  'lng': -3.6350217},
 'Vallecas, Palomeras Bajas (Distrito Puente de Vallecas. Madrid Capital)': {'lat': 40.384267,
  'lng': -3.6630031},
 'calle Gomez Acebo, San Andrés (Distrito Villaverde. Madrid Capital)': {'lat': 40.3430374,
  'lng': -3.7089705},
 'calle Canarias, Palos de Moguer (Distrito Arganzuela. Madrid Capital)': {'lat': 40.4018264,
  'lng': -3.6923389},
 'Fuenlabrada - Centro, Avanzada-La Cueva (Fuenlabrada)': {'lat': 40.2930236,
  'lng': -3.7857758},
 'calle Alcalde Henche de La Plata, Sanchinarro (Distrito Hortaleza. Madrid Capital)': {'lat': 40.48866599999999,
  'lng': -3.649591199999999},
 'calle Magnolias, Algete': {'lat': 40.5985993, 'lng': -3.5000946},
 'calle Mesón de Paredes, Embaja

In [41]:
data_re.head(100)

,bathrooms,description,floor,location1,location2,price,price_m2,rooms,size,location,position
0,1.0,Coslada junto al recinto ferial. Vivienda de 7...,4ª,Recinto Ferial,Casco Antiguo (Coslada),226.000,2.897,2.0,78,"Recinto Ferial, Casco Antiguo (Coslada)","{'lat': 40.43040939999999, 'lng': -3.5578887}"
1,1.0,"Bajos de 2 dormitorios 50m2, totalmente refor...",0,"calle Isletas, nº 1",Zona Estación (Coslada),98.500,1.790,2.0,55,"calle Isletas, nº 1, Zona Estación (Coslada)","{'lat': 40.4260459, 'lng': -3.5651646}"
2,1.0,Características del edificio:\n\n- Construido ...,NaN,"calle Mazo, nº 6",Pueblo Nuevo (Distrito Ciudad Lineal. Madrid C...,129.000,1.842,3.0,70,"calle Mazo, nº 6, Pueblo Nuevo (Distrito Ciuda...","{'lat': 40.43073589999999, 'lng': -3.6350217}"
3,1.0,REF: 12130 Vivienda exterior localizada en una...,NaN,Vallecas,Palomeras Bajas (Distrito Puente de Vallecas. ...,220.000,3.098,3.0,71,"Vallecas, Palomeras Bajas (Distrito Puente de ...","{'lat': 40.384267, 'lng': -3.6630031}"
4,1.0,"Excelente piso reformado en venta, de dos dorm...",0,calle Gomez Acebo,San Andrés (Distrito Villaverde. Madrid Capital),148.000,2.144,2.0,69,"calle Gomez Acebo, San Andrés (Distrito Villav...","{'lat': 40.3430374, 'lng': -3.7089705}"
5,1.0,"Tercera planta exterior con ascensor de 87 m2,...",NaN,calle Canarias,Palos de Moguer (Distrito Arganzuela. Madrid C...,300.000,3.448,3.0,87,"calle Canarias, Palos de Moguer (Distrito Arga...","{'lat': 40.4018264, 'lng': -3.6923389}"
6,1.0,"Se vende piso en zona de calle Málaga, 85 metr...",NaN,Fuenlabrada - Centro,Avanzada-La Cueva (Fuenlabrada),134.000,1.576,3.0,85,"Fuenlabrada - Centro, Avanzada-La Cueva (Fuenl...","{'lat': 40.2930236, 'lng': -3.7857758}"
7,2.0,"INMOLINE, 91 616 32 20 y whatsapp 673 125 727:...",NaN,calle Alcalde Henche de La Plata,Sanchinarro (Distrito Hortaleza. Madrid Capital),450.000,4.090,3.0,110,"calle Alcalde Henche de La Plata, Sanchinarro ...","{'lat': 40.48866599999999, 'lng': -3.649591199..."
8,2.0,SITUACION:\n\nLa vivienda se encuentra en la l...,NaN,calle Magnolias,Algete,166.990,2.036,2.0,82,"calle Magnolias, Algete","{'lat': 40.5985993, 'lng': -3.5000946}"
9,1.0,Vivienda en barrio de Lavapies. Ideal como inv...,0,calle Mesón de Paredes,Embajadores-Lavapiés (Distrito Centro. Madrid ...,140.000,4.000,2.0,35,"calle Mesón de Paredes, Embajadores-Lavapiés (...","{'lat': 40.4087204, 'lng': -3.7033913}"


In [44]:
dict_total = {**dict_pos, **dict_pos2}

In [45]:
data_re['position'] = data_re.loc[:5000, 'location'].apply(lambda x: get_value_from_dict(dict_total, x))

In [46]:
data_re.head(300)

,bathrooms,description,floor,location1,location2,price,price_m2,rooms,size,location,position
0,1.0,Coslada junto al recinto ferial. Vivienda de 7...,4ª,Recinto Ferial,Casco Antiguo (Coslada),226.000,2.897,2.0,78,"Recinto Ferial, Casco Antiguo (Coslada)","{'lat': 40.43040939999999, 'lng': -3.5578887}"
1,1.0,"Bajos de 2 dormitorios 50m2, totalmente refor...",0,"calle Isletas, nº 1",Zona Estación (Coslada),98.500,1.790,2.0,55,"calle Isletas, nº 1, Zona Estación (Coslada)","{'lat': 40.4260459, 'lng': -3.5651646}"
2,1.0,Características del edificio:\n\n- Construido ...,NaN,"calle Mazo, nº 6",Pueblo Nuevo (Distrito Ciudad Lineal. Madrid C...,129.000,1.842,3.0,70,"calle Mazo, nº 6, Pueblo Nuevo (Distrito Ciuda...","{'lat': 40.43073589999999, 'lng': -3.6350217}"
3,1.0,REF: 12130 Vivienda exterior localizada en una...,NaN,Vallecas,Palomeras Bajas (Distrito Puente de Vallecas. ...,220.000,3.098,3.0,71,"Vallecas, Palomeras Bajas (Distrito Puente de ...","{'lat': 40.384267, 'lng': -3.6630031}"
4,1.0,"Excelente piso reformado en venta, de dos dorm...",0,calle Gomez Acebo,San Andrés (Distrito Villaverde. Madrid Capital),148.000,2.144,2.0,69,"calle Gomez Acebo, San Andrés (Distrito Villav...","{'lat': 40.3430374, 'lng': -3.7089705}"
5,1.0,"Tercera planta exterior con ascensor de 87 m2,...",NaN,calle Canarias,Palos de Moguer (Distrito Arganzuela. Madrid C...,300.000,3.448,3.0,87,"calle Canarias, Palos de Moguer (Distrito Arga...","{'lat': 40.4018264, 'lng': -3.6923389}"
6,1.0,"Se vende piso en zona de calle Málaga, 85 metr...",NaN,Fuenlabrada - Centro,Avanzada-La Cueva (Fuenlabrada),134.000,1.576,3.0,85,"Fuenlabrada - Centro, Avanzada-La Cueva (Fuenl...","{'lat': 40.2930236, 'lng': -3.7857758}"
7,2.0,"INMOLINE, 91 616 32 20 y whatsapp 673 125 727:...",NaN,calle Alcalde Henche de La Plata,Sanchinarro (Distrito Hortaleza. Madrid Capital),450.000,4.090,3.0,110,"calle Alcalde Henche de La Plata, Sanchinarro ...","{'lat': 40.48866599999999, 'lng': -3.649591199..."
8,2.0,SITUACION:\n\nLa vivienda se encuentra en la l...,NaN,calle Magnolias,Algete,166.990,2.036,2.0,82,"calle Magnolias, Algete","{'lat': 40.5985993, 'lng': -3.5000946}"
9,1.0,Vivienda en barrio de Lavapies. Ideal como inv...,0,calle Mesón de Paredes,Embajadores-Lavapiés (Distrito Centro. Madrid ...,140.000,4.000,2.0,35,"calle Mesón de Paredes, Embajadores-Lavapiés (...","{'lat': 40.4087204, 'lng': -3.7033913}"


In [48]:
dict_pos3 = {}
for location in data_re['location'][5000:]:
    dict_pos3[location] = get_position(location)

In [49]:
dict_total = {**dict_pos3, **dict_total}

In [53]:
data_re['position'] = data_re.loc[:,'location'].apply(lambda x: get_value_from_dict(dict_total, x))

In [54]:
data_re

,bathrooms,description,floor,location1,location2,price,price_m2,rooms,size,location,position
0,1.0,Coslada junto al recinto ferial. Vivienda de 7...,4ª,Recinto Ferial,Casco Antiguo (Coslada),226.000,2.897,2.0,78,"Recinto Ferial, Casco Antiguo (Coslada)","{'lat': 40.43040939999999, 'lng': -3.5578887}"
1,1.0,"Bajos de 2 dormitorios 50m2, totalmente refor...",0,"calle Isletas, nº 1",Zona Estación (Coslada),98.500,1.790,2.0,55,"calle Isletas, nº 1, Zona Estación (Coslada)","{'lat': 40.4260459, 'lng': -3.5651646}"
2,1.0,Características del edificio:\n\n- Construido ...,NaN,"calle Mazo, nº 6",Pueblo Nuevo (Distrito Ciudad Lineal. Madrid C...,129.000,1.842,3.0,70,"calle Mazo, nº 6, Pueblo Nuevo (Distrito Ciuda...","{'lat': 40.43073589999999, 'lng': -3.6350217}"
3,1.0,REF: 12130 Vivienda exterior localizada en una...,NaN,Vallecas,Palomeras Bajas (Distrito Puente de Vallecas. ...,220.000,3.098,3.0,71,"Vallecas, Palomeras Bajas (Distrito Puente de ...","{'lat': 40.384267, 'lng': -3.6630031}"
4,1.0,"Excelente piso reformado en venta, de dos dorm...",0,calle Gomez Acebo,San Andrés (Distrito Villaverde. Madrid Capital),148.000,2.144,2.0,69,"calle Gomez Acebo, San Andrés (Distrito Villav...","{'lat': 40.3430374, 'lng': -3.7089705}"
5,1.0,"Tercera planta exterior con ascensor de 87 m2,...",NaN,calle Canarias,Palos de Moguer (Distrito Arganzuela. Madrid C...,300.000,3.448,3.0,87,"calle Canarias, Palos de Moguer (Distrito Arga...","{'lat': 40.4018264, 'lng': -3.6923389}"
6,1.0,"Se vende piso en zona de calle Málaga, 85 metr...",NaN,Fuenlabrada - Centro,Avanzada-La Cueva (Fuenlabrada),134.000,1.576,3.0,85,"Fuenlabrada - Centro, Avanzada-La Cueva (Fuenl...","{'lat': 40.2930236, 'lng': -3.7857758}"
7,2.0,"INMOLINE, 91 616 32 20 y whatsapp 673 125 727:...",NaN,calle Alcalde Henche de La Plata,Sanchinarro (Distrito Hortaleza. Madrid Capital),450.000,4.090,3.0,110,"calle Alcalde Henche de La Plata, Sanchinarro ...","{'lat': 40.48866599999999, 'lng': -3.649591199..."
8,2.0,SITUACION:\n\nLa vivienda se encuentra en la l...,NaN,calle Magnolias,Algete,166.990,2.036,2.0,82,"calle Magnolias, Algete","{'lat': 40.5985993, 'lng': -3.5000946}"
9,1.0,Vivienda en barrio de Lavapies. Ideal como inv...,0,calle Mesón de Paredes,Embajadores-Lavapiés (Distrito Centro. Madrid ...,140.000,4.000,2.0,35,"calle Mesón de Paredes, Embajadores-Lavapiés (...","{'lat': 40.4087204, 'lng': -3.7033913}"
